# Definiciones auxiliares

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Definiciones sobre árboles binarios

In [2]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

defined class Tree
defined class Empty
defined class Node

In [3]:
object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
    
    def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
        tree match {
            case Empty() => 
                empty
            case Node(left, a, right) =>
                node(foldTree(left)(empty)(node),
                    a,
                    foldTree(right)(empty)(node))
        }
}

import Tree._

defined object Tree
import Tree._

# Ejercicio

La función _penultimate_ devuelve el penúltimo elemento de una lista, en caso de que este exista. 

In [4]:
class TestPenultimate(
    penultimate: List[Int] => Option[Int]
) extends FlatSpec with Matchers{
    
    "penultimate" should "work" in {
        penultimate(List()) shouldBe None
        penultimate(1 :: Nil) shouldBe None
        penultimate(2 :: (3 :: Nil)) shouldBe Some(2)
        penultimate(1 :: List(2,4,3,5)) shouldBe Some(3)
    }
}

defined class TestPenultimate

Implementa la función de manera recursiva (no es necesario que sea recursiva por la cola).

In [5]:
def penultimate[A](list: List[A]): Option[A] = 
    list match {
        case Nil => None : Option[A]
        case head :: tail => 
            val tailSol: Option[A] = penultimate(tail)
            tail match {
                case Nil => None
                case head2 :: tail2 => 
                    tail2 match {
                        case Nil => Some(head)
                        case _ => tailSol
                    } 
            }
    }

defined function penultimate

In [10]:
def penultimate[A](list: List[A]): Option[A] = 
    list match {
        case Nil => None
        case _ :: Nil => None
        case head :: _ :: Nil => Some(head)
        case _ :: tail => penultimate(tail)
    }

defined function penultimate

In [11]:
def penultimate[A](list: List[A]): Option[A] = 
    list match {
        case Nil | _ :: Nil => None
        case head :: _ :: Nil => Some(head)
        case _ :: tail => penultimate(tail)
    }

defined function penultimate

Versión con foldRight

In [12]:
def penultimate[A](list: List[A]): Option[A] = 
    list.foldRight(None : Option[A])(
        (head: A, tailSol: Option[A]) => 
            tail match {
                case Nil => None
                case head2 :: tail2 => 
                    tail2 match {
                        case Nil => Some(head)
                        case _ => tailSol
                    } 
            }
    )
/*
    list match {
        case Nil => None : Option[A]
        case head :: tail => 
            val tailSol: Option[A] = penultimate(tail)
            tail match {
                case Nil => None
                case head2 :: tail2 => 
                    tail2 match {
                        case Nil => Some(head)
                        case _ => tailSol
                    } 
            }
    }
    */

cmd12.sc:4: not found: value t
            t match {
            ^Compilation Failed

: 

In [13]:
def penultimate[A](list: List[A]): Option[A] = 
    list.foldRight((Nil : List[A], None : Option[A]))(
        (head: A, tailSol: (List[A], Option[A])) => 
            tailSol._1 match {
                case Nil => (head :: tailSol._1, None)
                case head2 :: tail2 => 
                    tail2 match {
                        case Nil => (head :: tailSol._1, Some(head))
                        case _ => (head :: tailSol._1, tailSol._2)
                    } 
            }
    )._2


defined function penultimate

In [17]:
def penultimate[A](list: List[A]): Option[A] = 
    list.foldRight((Nil : List[A], None : Option[A])){
        case (head, (Nil, _)) => (head :: Nil, None)
        case (head, (head2 :: Nil, _)) => (head :: head2 :: Nil, Some(head))
        case (head, (tail, tailSol)) => (head :: tail, tailSol)
    }._2


defined function penultimate

Con foldRight, pero con una variable auxiliar distinta del resto de la lista: 
Primero con el patrón recursivo.

In [20]:
def penultimate[A](list: List[A]): Option[A] = {

    def aux[A](list: List[A]): (Int, Option[A]) = 
        list match {
            case Nil => (1, None : Option[A])
            case head :: tail => 
                val tailSol: (Int, Option[A]) = aux(tail)
                if (tailSol._1 == 1) (tailSol._1 + 1, None)
                else if (tailSol._1 == 2) (tailSol._1 + 1, Some(head))
                else (tailSol._1 + 1, tailSol._2)
        }
    
    aux(list)._2
}

defined function penultimate

Con foldright:

In [25]:
def penultimate[A](list: List[A]): Option[A] = 
    list.foldRight((1, None : Option[A])){
        case (head, (1, _)) => (2, None)
        case (head, (2, _)) => (3, Some(head))
        case (head, tailSol) => tailSol
    }._2

defined function penultimate

Mediante un algoritmo iterativo (recursión final)

None, None, 1 :: 2 :: 3 :: 4 :: 5 :: Nil

     None, S(1), 2 :: 3 :: 4 :: 5 :: Nil

          S(1), S(2), 3 :: 4 :: 5 :: Nil

               S(2), S(3), 4 :: 5 :: Nil
   
                    S(3), S(4), 5 :: Nil

                         S(4), S(5), Nil



In [28]:
def penultimate[A](list: List[A]): Option[A] = {
    def aux(out1: Option[A], out2: Option[A], l: List[A]): Option[A] = 
        l match {
            case Nil => out1
            case (e: A) :: tail => 
                aux(out2, Some(e), tail)
        }
    
    aux(None, None, list)
}

defined function penultimate

Lo transformamos para hacer la conversión a foldLeft más sencillamente: 

In [30]:
def penultimate[A](list: List[A]): Option[A] = {
    def aux(out: (Option[A], Option[A]), l: List[A]): (Option[A], Option[A]) = 
        l match {
            case Nil => out
            case (e: A) :: tail => 
                aux((out._2, Some(e)), tail)
        }
    
    aux((None, None), list)._1
}

defined function penultimate

Versión con foldLeft:

In [32]:
def penultimate[A](list: List[A]): Option[A] = 
    list.foldLeft((None: Option[A], None: Option[A])){
        case ((_, out2), e) => 
            (out2, Some(e))
    }._1

defined function penultimate

In [33]:
run(new TestPenultimate(penultimate))

cmd3$Helper$TestPenultimate:
penultimate
- should work


# Ejercicio

La función `odds` devuelve los elementos de una lista que se encuentran situados en una posición impar (donde la posición del primer elemento es uno, del segundo dos, etc.).

In [ ]:
class TestOdds(
    odds: List[Int] => List[Int]
) extends FlatSpec with Matchers{
    
    "odds" should "work" in {
        odds(List()) shouldBe List()
        odds(List(1)) shouldBe List(1)
        odds(List(2,3)) shouldBe List(2)
        odds(List(1,2,4,3,5)) shouldBe List(1,4,5)
        odds(List(1,2,4,3,5,6)) shouldBe List(1,4,5)
    }
}

In [ ]:
def odds[A](list: List[A]): List[A] = 
    ???

In [ ]:
run(new TestOdds(odds))

# Ejercicio


Un árbol se encuentra _balanceado_ si (1) es el árbol vacío o (2) si sus hijos izquierdo y derecho se encuentran balanceados, y la diferencia entre sus profundidades no excede la unidad. La profundidad de un árbol vacío es cero, y la de un árbol no vacío el máximo de las profundidades de sus hijos más uno.

In [36]:
class TestBalanced(
    balanced: Tree[Int] => Boolean
) extends FlatSpec with Matchers {
    
    "balanced" should "work" in {
        balanced(void) shouldBe true
        balanced(leaf(1)) shouldBe true
        balanced(left(leaf(1), 2)) shouldBe true
        balanced(node(leaf(1), 2, leaf(3))) shouldBe true
        balanced(node(left(leaf(1),4), 2, leaf(3))) shouldBe true
        balanced(right(3, left(leaf(1),4))) shouldBe false
        balanced(node(right(3, left(leaf(1),4)),5,leaf(3))) shouldBe false
    }
}

defined class TestBalanced

versión divide y vencerás, con patrón de recursión:

In [43]:
def height[A](tree: Tree[A]): Int = 
    tree match {
        case Empty() => 
            0
        case Node(left, root, right) => 
            val leftH: Int = height(left)
            val rightH: Int = height(right)
            (leftH max rightH) + 1
    }

defined function height

In [44]:
def balanced[A](tree: Tree[A]): Boolean = 
    tree match {
        case Empty() => 
            true
        case Node(left, root, right) => 
            val leftBalanced: Boolean = balanced(left)
            val rightBalanced: Boolean = balanced(right)
            leftBalanced && rightBalanced && (height(left) - height(right)).abs <= 1 : Boolean 
    }

defined function balanced

versión divide y vencerás, con foldTree: 

In [45]:
def balanced[A](tree: Tree[A]): Boolean = 
    foldTree(tree)(true){
        case (leftBalanced, root, rightBalanced) => 
            leftBalanced && rightBalanced && (height(l) - height(r)).abs <= 1 : Boolean 
    }

cmd45.sc:4: not found: value l
            leftBalanced && rightBalanced && (height(l) - height(r)).abs <= 1 : Boolean 
                                                     ^cmd45.sc:4: not found: value r
            leftBalanced && rightBalanced && (height(l) - height(r)).abs <= 1 : Boolean 
                                                                 ^Compilation Failed

: 

In [46]:
def balanced[A](tree: Tree[A]): Boolean = 
    foldTree(tree)((0, true)){
        case ((leftH, leftBalanced), root, (rightH, rightBalanced)) => 
            ((leftH max rightH)+ 1 : Int, 
              leftBalanced && rightBalanced && (leftH - rightH).abs <= 1 : Boolean)
    }._2

defined function balanced

In [47]:
run(new TestBalanced(balanced))

cmd35$Helper$TestBalanced:
balanced
- should work


In [48]:
def height[A](tree: Tree[A]): Int = 
    foldTree(tree)(0)(
        (leftH, _, rightH) =>
            (leftH max rightH) + 1
    )

defined function height